In [ ]:
import cv2
import numpy as np
import pygame
import os
import mediapipe as mp

# Initialize Pygame for audio playback
pygame.init()

# Path to the folder containing MP3 files
folder_path = "songs"

# List all MP3 files in the folder
mp3_files = [file for file in os.listdir(folder_path) if file.endswith('.mp3')]

# Initialize Pygame mixer
pygame.mixer.init()

# Initialize MediaPipe Hand model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize current file index
current_file_index = 0

# Function to play MP3 files
def play_mp3(file_index):
    pygame.mixer.music.load(os.path.join(folder_path, mp3_files[file_index]))
    pygame.mixer.music.play()

# Function to detect hand gestures
def detect_gesture(hand_landmarks):
    # Get coordinates of specific landmarks
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
    ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

    # Calculate distances between fingertips
    thumb_to_index_dist = np.sqrt((thumb_tip.x - index_finger_tip.x)**2 + (thumb_tip.y - index_finger_tip.y)**2)
    index_to_middle_dist = np.sqrt((index_finger_tip.x - middle_finger_tip.x)**2 + (index_finger_tip.y - middle_finger_tip.y)**2)
    middle_to_ring_dist = np.sqrt((middle_finger_tip.x - ring_finger_tip.x)**2 + (middle_finger_tip.y - ring_finger_tip.y)**2)
    ring_to_pinky_dist = np.sqrt((ring_finger_tip.x - pinky_tip.x)**2 + (ring_finger_tip.y - pinky_tip.y)**2)

    # Recognize gestures based on distances
    if thumb_to_index_dist < 0.03:
        return 1  # Gesture for moving to next file
    elif ring_to_pinky_dist < 0.03:
        return 2  # Gesture for moving to previous file
    else:
        return 0  # No gesture detected

# Main function
def main():
    global current_file_index  # Declare current_file_index as global

    # Open webcam
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB (MediaPipe requires RGB input)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect hand landmarks
        results = hands.process(rgb_frame)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks (for visualization)
                mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Detect hand gestures
                gesture = detect_gesture(hand_landmarks)

                # Control playback based on detected gesture
                if gesture == 1:
                    # Move to the next MP3 file
                    current_file_index = (current_file_index + 1) % len(mp3_files)
                    play_mp3(current_file_index)
                elif gesture == 2:
                    # Move to the previous MP3 file
                    current_file_index = (current_file_index - 1) % len(mp3_files)
                    play_mp3(current_file_index)

        # Display the frame
        cv2.imshow('Frame', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close all windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
